# **Projet Python 2A**
Jean-Baptiste Laval - Sirine Louati - Hadrien Lolivier

*Ce projet est réalisé dans le cadre du cours de Python de Lino Galiana pour l'année 2021-2022.*

<div align="center">
  <img src="https://www.planete-energies.com/sites/default/files/styles/media_full_width_940px/public/thumbnails/image/visuel_emissions_co2_small.jpg?itok=Rrh1f3Qy"  width="600"><br>
</div>

## Introduction

**L'objectif** de notre programme est de calculer l'empreinte carbone de différentes recettes d'un plat choisi par l'utilisateur. Il se décompose en quatre grandes étapes :

- Étape 1 : extraction de recettes en scrappant le site Marmiton
- Étape 2 : identification des ingrédients et calcul de leur empreinte carbone grâce à la base Agribalyse
- Étape 3 : agrégation des données
- Étape 4 : affichage synthétisé des résultats

## Structure du répertoire

À la racine du répertoire, on trouve :
- **`main.py`** : rassemble les principales fonctions du programme, notamment la fonction `main` qui permet d'exécuter le programme en entier.
- **`requirements.txt`** : liste des modules Python requis
- **`README.md`** : présentation détaillée du projet
- **`notebook.ipynb`** : visite guidée des principales fonctions du répertoire et exemples
- le dossier **`data`** dans lequel on trouve :
    - **`agribalyse.csv`** : c'est une copie locale de la base Agribalyse utilisée en guise de cache (si le fichier existe, alors le programme n'a pas besoin de le télécharger de nouveau, sinon le programme le télécharge).
    - **`interface.html`** : c'est le résultat final du programme (il est écrasé si le programme est exécuté avec de nouveaux arguments)
- le dossier **`src`** : c'est le dossier qui contient le code

## Installation

Les modules Python utilisés par le programme sont précisés dans le fichier **`requirements.txt`**. Il faut donc les installer.

In [ ]:
!pip install -r requirements.txt

Notre scrapper utilise le browser **`chromedriver`** qu'il est donc nécessaire d'installer également.

Ci-dessous figurent deux manières d'installer **`chromedriver`**, soit sous Ubuntu (valable aussi sur Google Colaboratory), soit sous MacOS. En cas de difficulté, se référer au [site](https://chromedriver.chromium.org/) de **`chromedriver`**.

In [ ]:
### Installation de chromedriver sous Ubuntu

!sudo apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!sudo chmod +x /usr/lib/chromium-browser/chromedriver
!export PATH=$PATH:/usr/lib/chromium-browser/chromedriver

In [ ]:
### Installation de chromedriver sous MacOS

!npm install chromedriver

## Fonctionnement global

Pour utiliser le programme, il suffit d'importer la fonction `main` du fichier **`main.py`**, qui prend en argument le nom d'une recette de cuisine `recipe` et le nombre maximal de recettes à extraire `nmax`. Le résultat est le fichier **`interface.html`** généré dans le dossier **`data`**, qui peut être ouvert dans n'importe quel navigateur.

In [2]:
from main import main

main(recipe = "gâteau au chocolat", nmax = 10)


Starting to scrap Marmiton... (please wait about 10 seconds)

Cookies handled !




 80%|███████████████████████████████████████████████████▏            | 8/10 [00:30<00:06,  3.00s/it]

[INFO] Cannot convert an ingredient to grams : (poire au sirop, {'quantity': '1 ', 'unit': 'boîte'}).                             



 90%|█████████████████████████████████████████████████████████▌      | 9/10 [00:33<00:03,  3.01s/it]

[INFO] Cannot convert an ingredient to grams : (1, {'quantity': '', 'unit': 'noix de coco rapée (quantité selon vos goûts)'}).                             



100%|███████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.72s/it]



The following ingredients were missing from the conversion list (conversions.py) :
    ---> 'moules à muffin ou des caissettes en papier' was remplaced by 'pomm de terr'
    ---> 'sucre glace' was remplaced by 'sucr'
    ---> 'noix' was remplaced by 'poir'
    ---> 'lait de coco' was remplaced by 'lait'

These ingredients shall be added to the conversion list to be properly processed in the future.

Marmiton's data is extracted !


Processing ingredients...



100%|███████████████████████████████████████████████████████████████| 22/22 [00:26<00:00,  1.21s/it]



    ###################################################################
    ### L'interface a été générée avec succès : data/interface.html ###
    ###################################################################
    


## Étape 1: extraction de recettes en scrappant le site Marmiton

Le fonction qui réalise le scrapping est la fonction `marmiton_scrapper` du fichier **`scrapper_marmiton.py`** du dossier **`src`**.

**Arguments**

Elle prend en argument le nom d'une recette et un nombre maximal de recettes à extraire.


**Sorties**

Elle renvoie un dictionnaire à 3 niveaux de profondeur, référençant les différentes recettes trouvées, avec les informations correspondant à chacune d'entre elles (nom, url, nombre de commentaires, note, nombre de portions...), et pour chacune d'entre elles un dictionnaire d'ingrédients, associant à chaque ingrédient sa masse en grammes.


**Difficultés rencontrées**

L'une des premières difficultés survenues lors de l'implémentation du scrapper fut les blocages d'accès au site Marmiton. Nous avons donc eu recours à la fonction `sleep` pour limiter la fréquence des requêtes au serveur. Par ailleurs, nous avons dû surmonter certaines difficultés inattendues comme l'apparition potentielle de cookies (qui peut bloquer l'exécution du scrapper) ou encore les mises à jour du site qui peuvent rapidement rendre obsolètes notre scrapper (ce problème a été résolu en caractérisant plus intelligemment les champs html à extraire).


**Limites du scrapper**

Dans la partie scrapping, certaines simplifications ont été nécessaires pour mener à bien notre projet. Tout d'abord, nous avons fait le choix de nous restreindre aux recettes issues du site Marmiton, du fait de la variété de recettes associées à chaque plat. Cependant, il aurait pu être intéressant d'utiliser des recettes de différents site ; mais pour des raisons de simplicité nous nous sommes concentrés sur les recettes proposées par Marmiton.
De plus, notre scrapper se devait de renvoyer un dictionnaire dans lequel chaque ingrédient était associé à la masse indiquée dans la recette. Il a donc fallu procéder à des conversions et à certaines simplifications pour rendre compte de la masse de certaines quantités qui ne sont pas naturellement exprimées en grammes, par exemple les ingrédients du type "1 sachet de levure", "2 cuillers à café d'huile d'olive" ou "3 pommes Golden". Nous avons donc utilisé d'une part une fonction de normalisation du texte (fonction `clean_string` dans le fichier **`utils.py`** du dossier **`src`**) et d'autre part une table de conversion dans le fichier **`conversions.py`** du dossier **`src`**.


**Dummy output**

Si le scrapper ne parvient pas à s'exécuter, une sortie de démonstration `dummy_output1` est disponible dans le fichier **`dummy.py`** du dossier **`src`**, grâce à laquelle il est possible d'exécuter malgré tout les autres fonctions du programme.

In [14]:
from src.scrapper_marmiton import marmiton_scrapper
from pprint import pprint  # only to improve the display of the ouputs

output_scrapper = marmiton_scrapper(recipe_name="gâteau au chocolat", n=5)

pprint(output_scrapper)


Starting to scrap Marmiton... (please wait about 10 seconds)

Cookies handled !




100%|█████████████████████████████████████████████████████████████████| 5/5 [00:23<00:00,  4.79s/it]



The following ingredients were missing from the conversion list (conversions.py) :
    ---> 'moules à muffin ou des caissettes en papier' was remplaced by 'pomm de terr'
    ---> 'sucre glace' was remplaced by 'sucr'

These ingredients shall be added to the conversion list to be properly processed in the future.

Marmiton's data is extracted !

{'Gâteau au chocolat (seulement pour les enfants!)': {'ingredients': {'beurre': 50.0,
                                                                      'crème dessert au chocolat': 180.0,
                                                                      'farine': 137.5,
                                                                      'levure': 5.5,
                                                                      'oeufs': 350.0,
                                                                      'sucre': 397.5,
                                                                      'sucre vanillé': 11.0},
                      

## Etape 2: identification des ingrédients et calcul de leur empreinte carbone grâce à la base Agribalyse

À partir de la sortie Marmiton obtenue via le scrapper, comment peut-on obtenir l'empreinte carbone d'une recette ?

C'est l'objectif de la deuxième étape ! Elle consiste à chercher les ingrédients de la base Agribalyse qui correspondent le mieux aux ingrédients de notre recette. Les fonctions associées à cette étape se trouvent dans le fichier **`food2emissions.py`** du dossier **`src`**.

Il s'agit tout d'abord d'importer et de nettoyer la base de données d'Agribalyse.

In [3]:
from src.food2emissions import import_data_from_agribalyse

products_data = import_data_from_agribalyse()

products_data.head()

,name_prod,dqr,agriculture_co2,transformation_co2,packaging_co2,transport_co2,distribution_co2,consumption_co2,clean_name_prod
0,Abondance,2.24,5.327912,0.369609,0.266627,0.265165,0.029051,0.004799,abond
1,"Abricot au sirop léger, appertisé, égoutté",2.46,0.188315,0.166137,0.350156,0.225336,0.051200,0.239977,abricot au sirop leg appertis egoutt
2,"Abricot au sirop léger, appertisé, non égoutté",2.46,0.117508,0.103670,0.218498,0.140610,0.031949,0.000000,abricot au sirop leg appertis non egoutt
3,"Abricot au sirop, appertisé, égoutté",2.46,0.188315,0.166137,0.350156,0.225336,0.051200,0.239977,abricot au sirop appertis egoutt
4,"Abricot au sirop, appertisé, non égoutté",2.44,0.117508,0.103670,0.218498,0.140610,0.031949,0.000000,abricot au sirop appertis non egoutt


Ensuite, il s'agit de trouver chaque ingrédient dans la base. Or, parfois la dénomination d'un ingrédient peut changer. Il faut donc trouver dans la base le produit le plus "proche" d'un ingrédient donné.

Dans ce but, nous nous sommes interrogé sur la meilleure manière de mesurer la distance entre deux mots.
Nous avons donc d'une part travaillé sur la normalisation des mots grâce à la fonction `clean_string` du fichier **`utils.py`** du dossier **`src`** qui met les mots en minuscules, retire les caractères qui ne sont pas des lettres ou des espaces, retire les espaces en début et fin de chaîne de caractères, retire les accents et utilise du _stemming_ pour éviter les différences d'accord en nombre (singulier et pluriel).
D'autre part, nous avons testé 4 méthodes de calcul de distance entre deux mots. Afin de mieux pouvoir les comparer, nous avons normalisé toutes les distances entre 0 et 1 : 0 lorsque deux mots sont très proches, 1 lorsqu'ils sont très différents.

La fonction `compare_methods` permet de visualiser pour une liste de mots-tests, les `n` mots les plus proches dans la base d'Agribalyse selon chacune des distances. Chaque distance est identifiée par un trigramme.

La distance la plus efficace nous semble être la distance dite "personnalisée" `PER` que nous avons nous-mêmes implémentée.

In [5]:
from src.food2emissions import compare_methods

compare_methods(["chocolat noir", "sucre en poudre"], products_data, n=5)




----- CHOCOLAT NOIR -----
----- chocolat noir -----

GPM
                    name_prod         clean_name_prod  distance
491           Chipolata, crue            chipolat cru  0.360000
500   Chocolat au lait fourré  chocolat au lait fourr  0.371429
492          Chipolata, cuite           chipolat cuit  0.384615
536         Choucroute garnie          choucrout garn  0.407407
505  Chocolat blanc, tablette   chocolat blanc tablet  0.411765


FWZ
                                             name_prod  ... distance
512             Chocolat noir fourré praliné, tablette  ...      0.0
513  Chocolat noir sans sucres ajoutés, avec édulco...  ...      0.0
511       Chocolat noir fourrage confiseur à la menthe  ...      0.0
510  Chocolat noir aux fruits secs (noisettes, aman...  ...      0.0
509  Chocolat noir aux fruits (orange, framboise, p...  ...      0.0

[5 rows x 3 columns]


LEV
                    name_prod         clean_name_prod  distance
492          Chipolata, cuite           chip

Enfin, la fonction `compute_emissions` renvoie les données correspondant au produit de la base d'Agribalyse le plus proche d'un ingrédient donné.

In [8]:
from src.food2emissions import compute_emissions

compute_emissions("chocolat noir", products_data).head()

,name_prod,dqr,agriculture_co2,transformation_co2,packaging_co2,transport_co2,distribution_co2,consumption_co2,clean_name_prod,uncertainty,agribalyse_match
512,chocolat noir,3.34,16.761333,1.274366,0.10292,0.411703,0.014105,0.0,chocolat noir fourr pralin tablet,0.121212,"Chocolat noir fourré praliné, tablette"


**Limites**

Malgré l'utilisation d'une distance personnalisée, on observe toujours une incertitude au niveau du matching entre les ingrédients de la base d'Agribalyse et ceux de la recette Marmiton. Par exemple, pour l'ingrédient "eau", l'ingrédient de la base d'Agribalyse qui correspond le mieux est l'ingrédient "eau de vie" qui ne correspond pas exactement à ce qu'on attend :) Une piste d'amélioration serait potentiellement d'introduire de nouvelles distances, d'utiliser du NLP pour optimiser le matching, ou plus simplement de compléter la base d'Agribalyse.

## Etape 3: agrégation des données

Cette étape consiste à agréger les différentes sources de données. Elle se déroule dans le fichier **`aggregate.py`** du dossier **`src`**. La fonction principale est `aggregate_data`.

Cette fonction agrège 3 dataframes différents : `products_data`, `recipes` et `ingredients`.

- `products_data` est le dataframe obtenu à l'import des données d'Agribalyse (voir étape 2).

- `recipes` est le dataframe obtenu en convertissant le dictionnaire renvoyé par le scrapper en dataframe (voir étape 1).

- `ingredients` permet d'améliorer l'estimation de l'empreinte carbone des ingrédients, nous en détaillons le principe dans le fichier **`README.md`** à la racine du répertoire.

In [3]:
from src.aggregate import aggregate_data

aggregated_data = aggregate_data(output_scrapper)

aggregated_data.head()


Processing ingredients...



100%|███████████████████████████████████████████████████████████████| 16/16 [00:15<00:00,  1.05it/s]


,recipes,ingredients,weights,nb_comments,mark,url,agribalyse_match,distance,dqr,emission_type,co2
0,Gâteau au chocolat fondant rapide,chocolat pâtissier,0.303030,1439,4.7,https://www.marmiton.org/recettes/recette_gate...,"Chocolat blanc, tablette",0.123810,2.29,agriculture_co2,18.453596
1,Gâteau au chocolat fondant rapide,farine,0.075758,1439,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
2,Gâteau au chocolat des écoliers,farine,0.109170,1241,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
3,Petits gâteaux tout chocolat,farine,0.114943,337,4.8,https://www.marmiton.org/recettes/recette_peti...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997
4,Gâteau au chocolat (seulement pour les enfants!),farine,0.121520,339,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,0.116667,2.43,agriculture_co2,0.558997


## Étape 4 : affichage synthétisé des résultats

L'objectif de cette dernière partie est de visualiser graphiquement l'empreinte carbone de tous les ingrédients des recettes correspondant au plat donné en entrée. Pour cela, nous créons une interface graphique grâce au module `plotly` constituée de deux figures :
- la première figure représente l'empreinte carbone de chaque recette, avec les contributions carbone respectives de chaque ingrédient. En survolant les éléments avec la souris, on peut obtenir des renseignements complémentaires.
- la deuxième figure représente l'empreinte carbone de chaque ingrédient utilisé par au moins une des recettes, et précise la contribution respective de chacun des six postes d'émissions fournis par la base d'Agribalyse : agriculture, transformation, emballage, transport, distribution et consommation.

La fonction `preprocess_data` permet de mettre en forme les données.

In [4]:
from src.interface import preprocess_data

preprocessed_data = preprocess_data(aggregated_data)

preprocessed_data.head()

,Recette,Ingrédient,Masse (g),Nombre de commentaires,Note de la recette,Lien,Meilleure correspondance Agribalyse,Fiabilité de la correspondance Agribalyse (%),"Qualité des données (1 : très fiable, 5 : peu fiable)",Type d'émissions de CO2,Émissions de CO2 (kgCO2eq)
0,Gâteau au chocolat fondant rapide,Chocolat pâtissier,303.0,1439,4.7,https://www.marmiton.org/recettes/recette_gate...,"Chocolat blanc, tablette",88.0,2.29,Agriculture,18.45
1,Gâteau au chocolat fondant rapide,Farine,76.0,1439,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,88.0,2.43,Agriculture,0.56
2,Gâteau au chocolat des écoliers,Farine,109.0,1241,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,88.0,2.43,Agriculture,0.56
3,Petits gâteaux tout chocolat,Farine,115.0,337,4.8,https://www.marmiton.org/recettes/recette_peti...,Farine d'orge,88.0,2.43,Agriculture,0.56
4,Gâteau au chocolat (seulement pour les enfants!),Farine,122.0,339,4.7,https://www.marmiton.org/recettes/recette_gate...,Farine d'orge,88.0,2.43,Agriculture,0.56


Les fonctions `compare_recipes` et `compare_ingredients` renvoient donc des objets `plotly express` correspondant aux deux figures citées précédemment.

In [10]:
from src.interface import compare_recipes

compare_recipes(preprocessed_data).show()

In [11]:
from src.interface import compare_ingredients

compare_ingredients(preprocessed_data).show()

La fonction `interface` permet de combiner ces deux figures sur une même page html, automatiquement générée dans le dossier **`data`** sous le nom de **`interface.html`**. Il est possible d'ouvrir ce fichier avec n'importe quel navigateur.

In [9]:
from src.interface import interface

interface(preprocessed_data)


    ###################################################################
    ### L'interface a été générée avec succès : data/interface.html ###
    ###################################################################
    
